In [27]:
using Zygote

In [12]:
struct Parm
    t::Float64
    V::Float64
    λ::Float64
end

In [13]:
sigma = [[1.0 0.0; 0.0 1.0], [0.0 1.0; 1.0 0.0], [0.0 -1.0im; 1.0im 0.0], [1.0 0.0; 0.0 -1.0]]

4-element Vector{Matrix{ComplexF64}}:
 [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 [0.0 + 0.0im 1.0 + 0.0im; 1.0 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im -0.0 - 1.0im; 0.0 + 1.0im 0.0 + 0.0im]
 [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

In [88]:
a1 = [1.0, 0.0, 0.0]
a2 = [0.0, 1.0, 0.0]
a3 = [0.0, 0.0, 1.0]

3-element Vector{Float64}:
 0.0
 0.0
 1.0

In [31]:
function H(kx, ky, p::Parm)
    e0 = -p.t*(cos(kx))
    sz = -p.t*(cos(ky))
    sx = p.V + p.λ*sin(kx)
    sy = p.λ*sin(ky)
    hv = [e0, sx, sy, sz]
    #h::Array{ComplexF64,2} = hv' * sigma
    return hv
end

H (generic function with 3 methods)

In [42]:
function test_H(k::Float64, p::Parm)
    e0 = -p.t*(cos(k))
    sz = -p.t*(cos(k))
    sx = p.V + p.λ*sin(k)
    sy = p.λ*sin(k)
    hv = [e0, sx, sy, sz]
    h::Array{ComplexF64,2} = hv' * sigma
    return h
end

test_H (generic function with 1 method)

In [19]:
p0 = Parm(1.0, 0.2, 0.4)

Parm(1.0, 0.2, 0.4)

In [44]:
k = [1.0, 0.2, 0.5]

3-element Vector{Float64}:
 1.0
 0.2
 0.5

In [78]:
#Vx(k0::Vector{Float64},p::Parm) = 
#Vx(k1::Vector{Float64},p1::Parm) = 
V1 = Zygote.gradient(kx -> H([k0,k[2],k[3]],p), k0)[1]

4-element Vector{Float64}:
 0.19866933079506122
 0.39202663113649666
 0.0
 0.0

In [39]:
function set_vx(kx, ky,p0::Parm)
    gg_x = Zygote.jacobian(kx -> H(kx,ky,p0), kx)[1]
    #kk = k[1]
    #gg_x = (jacobian(kk -> H([kk,k[2],k[3]],p), kk)[1])
    #Vx::Array{ComplexF64,2} = gg_x' * sigma
    return gg_x
end

set_vx (generic function with 2 methods)

In [34]:
set_vx(k,p0)

4-element Vector{Float64}:
 0.8414709848078965
 0.2161209223472559
 0.0
 0.0

In [40]:
function set_vxx(kx, ky, p0::Parm)
    #kx=k[1]
    #ky=k[2]
    gg_x = (Zygote.jacobian(k -> set_vx(kx, ky, p0), k)[1])[:,1]
    #kk = k[1]
    #gg_x = (jacobian(kk -> H([kk,k[2],k[3]],p), kk)[1])
    #Vx::Array{ComplexF64,2} = gg_x' * sigma
    return gg_x
end

set_vxx (generic function with 2 methods)

In [41]:
set_vxx(k[1], k[2], p0)

ErrorException: Mutating arrays is not supported -- called copyto!(SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [25]:
m(u) = H(u, p0)
#H(x::Vector{Float64},p)


m (generic function with 2 methods)

In [15]:
using ForwardDiff

In [24]:
function H(k,p::Parm)
    e0 = -p.t*(cos(k[1])+cos(k[2]))
    sz = -p.t*(cos(k[3]))
    sx = p.V + p.λ*sin(k[1])
    sy = p.λ*sin(k[2])
    hv = [e0, sx, sy, sz]
    #h::Array{ComplexF64,2} = hv' * sigma
    return hv
end

H (generic function with 2 methods)

In [47]:
function set_vx_f(k,p::Parm)
    m2(u) = H(u, p)
    vx = ForwardDiff.jacobian(m2,k)[:,1]
end

set_vx_f (generic function with 1 method)

In [48]:
vx = set_vx_f(k,p0)

4-element Vector{Float64}:
 0.8414709848078965
 0.2161209223472559
 0.0
 0.0

In [49]:
function set_vxx_f(k,p::Parm)
    m2(u) = set_vx_f(u, p)
    vx = ForwardDiff.jacobian(m2,k)[:,1]
end

set_vxx_f (generic function with 1 method)

In [50]:
vxx = set_vxx_f(k,p0)

4-element Vector{Float64}:
  0.5403023058681398
 -0.33658839392315865
  0.0
  0.0

In [18]:
function g(u,up)
  u .* up
end

# パラメータの実際は vp として、
vp = collect(1.0:1.0:5.0);

# 関数の引数は v に格納して、
v = randn(5);

# 形式的に引数を一つだけ持つ関数 f を定義して、
f(u) = g(u, vp)

# 位置 v での f の Jacobian を計算してみると、
ForwardDiff.jacobian(f, v)

5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0
 0.0  2.0  0.0  0.0  0.0
 0.0  0.0  3.0  0.0  0.0
 0.0  0.0  0.0  4.0  0.0
 0.0  0.0  0.0  0.0  5.0

In [132]:
ForwardDiff.jacobian(hh, k)

MethodError: MethodError: no method matching hh(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(hh), Float64}, Float64, 3}})
Closest candidates are:
  hh(!Matched::Vector{Float64}) at ~/Documents/Codes/julia/nonlinear/test_zygote.ipynb:1

In [95]:
function VtoM(v::Vector{Float64})
    M = v' * sigma
    return M
end

VtoM (generic function with 1 method)

In [119]:
VVx = set_vx_f(k,p)

MethodError: MethodError: no method matching hh(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(hh), Float64}, Float64, 3}})
Closest candidates are:
  hh(!Matched::Vector{Float64}) at ~/Documents/Codes/julia/nonlinear/test_zygote.ipynb:1

In [105]:
function set_vxx_z(k::Vector{Float64},p::Parm)
    gg_x = (jacobian(k -> set_vx(k,p), k)[1])[:,1]
    #kk = k[1]
    #gg_x = (jacobian(kk -> H([kk,k[2],k[3]],p), kk)[1])
    #Vx::Array{ComplexF64,2} = gg_x' * sigma
    return gg_x
end

set_vxx_z (generic function with 1 method)

In [106]:
using ForwardDiff

In [110]:
function set_vxx_f(k::Vector{Float64},p::Parm)
    vx(k) = set_vx(k,p)
    gg_x = ForwardDiff.jacobian(vx, k)[1]
    #kk = k[1]
    #gg_x = (jacobian(kk -> H([kk,k[2],k[3]],p), kk)[1])
    #Vx::Array{ComplexF64,2} = gg_x' * sigma
    return gg_x
end

set_vxx_f (generic function with 1 method)

In [111]:
Vxx = set_vxx_f(k,p)

MethodError: MethodError: no method matching set_vx(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{var"#vx#120"{Parm}, Float64}, Float64, 3}}, ::Parm)
Closest candidates are:
  set_vx(!Matched::Vector{Float64}, ::Parm) at ~/Documents/Codes/julia/nonlinear/test_zygote.ipynb:1